In [28]:
import random
import numpy as np
import torch
import os
import pathlib
import pickle
from gurobipy import *

In [29]:
# data_generation_process = "SPO_Data_Generation"
data_generation_process = "DDR_Data_Generation"

# Parameters

In [30]:
# import pyepo
# generate data
grid = (3,3) # grid size
num_train = 100 # number of training data
num_feat = 5 # size of feature
num_test = 10000
deg = 1.2 # polynomial degree
e = 1 # scale of normal std or the range of uniform. For the error term

lower = 0 # coef lower bound
upper = 1 # coef upper bound
p = num_feat # num of features
d = 12 # num of arcs
alpha = e # scale of normal std or the range of uniform. For the error term
mis = deg # model misspecification


In [31]:
current_directory = os.getcwd()
parent_directory = os.path.dirname(current_directory)
grandparent_directory = os.path.dirname(parent_directory)
DataPath = os.path.dirname(grandparent_directory) + '/Data_New_Formulation/' + data_generation_process + "/"
pathlib.Path(DataPath).mkdir(parents=True, exist_ok=True)
print("grandparent_directory:", grandparent_directory)
print("DataPath:", DataPath)

DataPath = DataPath + "data_size="+str(num_train)+"_deg="+str(deg)+"_e="+str(e)+"/"
pathlib.Path(DataPath).mkdir(parents=True, exist_ok=True)

grandparent_directory: /Users/zhangxun/Dropbox/Research/Decision_Driven_Regularization/Code_MacBook
DataPath: /Users/zhangxun/Dropbox/Research/Decision_Driven_Regularization/Data_New_Formulation/DDR_Data_Generation/


# Generate Data

In [32]:
def Prepare_Data(DataPath,lower, upper, p, d, coef_seed,seed_all,num_test, num_train, alpha,mis,data_generation_process):
# #  ****** Coef generation *********
    from Data import data_generation
    data_gen = data_generation()
    # print("W_star = ",W_star[0,:])
    W_star = data_gen.generate_truth(DataPath,lower, upper, p, d, coef_seed,version = 0) 

    for seed in seed_all:
        DataPath_seed = DataPath +"Seed="+str(seed)+"/"
        pathlib.Path(DataPath_seed).mkdir(parents=True, exist_ok=True)
        # #  ****** Data generation *********
        x_test, c_test, x_train, c_train, W_star = data_gen.generate_samples(seed,DataPath_seed,p, d, num_test, num_train, alpha, W_star, mis, thres = 10, 
                                version = data_generation_process, x_dist = 'normal', e_dist = 'normal', x_low = 0, x_up = 2, x_mean = 2, x_var = 0.25, bump = 0) 
        # print()
    return x_test, c_test, x_train, c_train, W_star

In [33]:
coef_seed = 1
seed_all = [1]
x_test, c_test, x_train, c_train, W_star = Prepare_Data(DataPath,lower, upper, p, d, coef_seed,seed_all,num_test, num_train, alpha,mis,data_generation_process)

# Obtain OLS estimation

In [34]:
from OLS import ols_method
ols_method_obj = ols_method()
W_ols, w0_ols, t_ols, obj_ols = ols_method_obj.ols_solver("",x_train, c_train)

In [35]:
c_ols_esti = np.zeros((num_train,12))
for i in range(num_train):
    for j in range(12):
        c_ols_esti[i,j] = sum([W_ols[j][k] * x_train[i,k] for k in range(num_feat)])+ w0_ols[j]

In [37]:
c_ols_esti[0,:]

array([3.22394983, 3.56254114, 4.66443909, 4.62700498, 9.02172889,
       4.73081283, 6.75095528, 6.82448398, 6.54022279, 4.39169932,
       3.80641735, 4.38295283])

# Obtain DDR estimation

# Solve Shortest Path

In [38]:
def _getArcs(grid):
    arcs = []
    for i in range(grid[0]):
        # edges on rows
        for j in range(grid[1] - 1):
            v = i * grid[1] + j
            arcs.append((v, v + 1))
        # edges in columns
        if i == grid[0] - 1:
            continue
        for j in range(grid[1]):
            v = i * grid[1] + j
            arcs.append((v, v + grid[1]))
    return arcs

In [39]:
arcs = _getArcs(grid)

In [40]:
def solve_Shortest_Path(arcs,cost,grid):

    import gurobipy as gp
    from gurobipy import GRB
    # ceate a model
    m = gp.Model("shortest path")
    m.setParam('OutputFlag', 0)
    # varibles
    x = m.addVars(arcs, name="x")
    # sense
    # m.modelSense = GRB.MINIMIZE
    # flow conservation constraints
    for i in range(grid[0]):
        for j in range(grid[1]):
            v = i * grid[1] + j
            expr = 0
            for e in arcs:
                # flow in
                if v == e[1]:
                    expr += x[e]
                # flow out
                elif v == e[0]:
                    expr -= x[e]
            # source
            if i == 0 and j == 0:
                m.addConstr(expr == -1)
            # sink
            elif i == grid[0] - 1 and j == grid[0] - 1:
                m.addConstr(expr == 1)
            # transition
            else:
                m.addConstr(expr == 0)
    m.setObjective( sum([cost[ind] * x[arcs[ind]] for ind in range(len(arcs))]) , GRB.MINIMIZE)
    m.optimize()
    sol = m.getAttr('x')
    # print("sol = ",sol)
    # print("shortest_path = ",shortest_path)
    return sol,m.objVal

In [15]:
sol_oracle_dict = {}; obj_oracle_dict = {}
for j in range(num_train):
    sol_oracle_dict[j],obj_oracle_dict[j] = solve_Shortest_Path(arcs,c_train[j,:],grid)

In [47]:
np.shape(W_ols)

(12, 5)

In [53]:
diff = np.zeros(num_train)
for i in range(num_train):
    expr = 0
    for j in range(len(arcs)):
        # Compute the linear combination for row i and column j: (XW)_{ij} + w0_j.
        expr = expr + sol_oracle_dict[i][j] * (sum( [x_train[i, l] * W_ols[j][l] for l in range(num_feat)] ) + w0_ols[j])
    diff[i] = obj_oracle_dict[i] - expr

In [57]:
diff @ diff

np.float64(472.2127856471093)

In [ ]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np

# -------------------------------------
# Data Setup (example data)
# -------------------------------------
# n: number of observations (rows in X and D)
# p: number of features (columns in X, rows in W)
# k: number of responses (columns in D, and dimension of the output)

n, p, k = 100, 10, 5
np.random.seed(0)

# D is an n x k matrix
D = np.random.randn(n, k)

# X is an n x p matrix
X = np.random.randn(n, p)

# c is assumed to be a 1 x k vector for compatibility with the constraint
c = np.random.randn(k)

# y is an n-dimensional vector (n x 1)
y = np.random.randn(n)

# Gamma is a positive scalar for the constraint bound.
gamma = 1.0
num_acrs = len(arcs)
# -------------------------------------
# Build the Gurobi Model
# -------------------------------------
model = gp.Model("Matrix_Optimization_with_Intercept")

# Decision variables:
# W is a p x k matrix of decision variables.
W = model.addVars(num_acrs, num_train, name="W")
# w0 is a 1 x k vector (intercept), which will be added to each row.
w0 = model.addVars(num_acrs, name="w0")

# -------------------------------------
# Set the Objective
# -------------------------------------
# The objective is to minimize the squared Frobenius norm:
#   sum_{i=0}^{n-1} sum_{j=0}^{k-1} ( D[i,j] - (sum_{l=0}^{p-1} X[i,l]*W[l,j] + w0[j]) )^2.
obj_expr = gp.QuadExpr()
for i in range(num_train):
    for j in range(num_acrs):
        # Compute the linear combination for row i and column j: (XW)_{ij} + w0_j.
        expr = gp.quicksum( x_train[i, l] * W[l, j] for l in range(num_feat) ) + w0[j]
        diff = c_train[i, j] - expr
        obj_expr.add(diff * diff)
        5
model.setObjective(obj_expr, GRB.MINIMIZE)

# -------------------------------------
# Add the Norm Constraint
# -------------------------------------
# The constraint is:
#    || c - y^T (XW + repmat(w0, n, 1)) ||_2 <= gamma.
# Note that (XW + w0) is an n x k matrix.
# Then y^T (XW + w0) is a 1 x k vector, whose j-th component is:
#    t_j = sum_{i=0}^{n-1} y[i]*( (XW)[i,j] + w0[j] ).
# We enforce that sum_{j=0}^{k-1} ( c[j] - t_j )^2 <= gamma^2.

cons_expr = gp.QuadExpr()
for i in range(num_train):
    # Compute t_j = sum_{i=0}^{n-1} y[i] * ( (XW)[i,j] + w0[j] )
    t_i = gp.quicksum( sol_oracle_dict[i][j] * ( gp.quicksum( x_train[i, l] * W[l, j] for l in range(num_feat) ) + w0[j] )
                       for j in range(num_acrs) )
    diff = obj_oracle_dict[i] - t_i
    cons_expr.add(diff * diff)

model.addQConstr(cons_expr <= gamma**2, name="norm_constraint")

# -------------------------------------
# Optimize the Model
# -------------------------------------
model.optimize()

# -------------------------------------
# Retrieve and Print the Solution
# -------------------------------------
if model.status == GRB.OPTIMAL:
    # Retrieve optimal W as a p x k matrix
    W_opt = np.array([[W[l,j].X for j in range(k)] for l in range(p)])
    # Retrieve optimal intercept w0 as a vector of length k
    w0_opt = np.array([w0[j].X for j in range(k)])
    
    print("Optimal solution found:")
    print("W =\n", W_opt)
    print("w0 =", w0_opt)
    print("Objective value =", model.objVal)
else:
    print("No optimal solution found.")


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.6.0 23H420)

CPU model: Apple M1 Pro
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 0 rows, 55 columns and 0 nonzeros
Model fingerprint: 0xccdbb91a
Model has 330 quadratic objective terms
Model has 1 quadratic constraint
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  QMatrix range    [2e-02, 7e+02]
  QLMatrix range   [4e-03, 4e+01]
  Objective range  [1e-01, 4e+01]
  QObjective range [1e+00, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [0e+00, 0e+00]
  QRHS range       [3e-01, 3e-01]
Presolve time: 0.00s
Presolved: 63 rows, 119 columns, 505 nonzeros
Presolved model has 2 second-order cone constraints
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 1.671e+03
 Factor NZ  : 2.016e+03
 Factor Ops : 8.534e+04 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal    